In [23]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb langchain

In [24]:
import google.generativeai as genai
from google.colab import userdata
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from IPython.display import display
from IPython.display import Markdown
import textwrap


In [25]:
df=pd.read_csv("masked_dataset") #obtained from Masking code

NameError: name 'pd' is not defined

In [ ]:
GOOGLE_API_KEY=userdata.get('GEMINI')

In [ ]:

llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,
                             temperature=0.2,convert_system_message_to_human=True)

In [ ]:
text_store = 'test.txt'
loader = TextLoader(text_store)
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20, separators=['QUESTION'])
docs = splitter.split_documents(pages)

In [ ]:
docs[3]

Document(page_content='QUESTION: A pulmonary autopsy specimen from a 58-year-old woman who died of acute hypoxic respiratory failure was examined. She had recently undergone surgery for a fractured femur 3 months ago. Initial hospital course was uncomplicated, and she was discharged to a rehab facility in good health. Shortly after discharge home from rehab, she developed sudden shortness of breath and had cardiac arrest. Resuscitation was unsuccessful. On histological examination of lung tissue, fibrous connective tissue around the lumen of the pulmonary artery is observed. Which of the following is the most likely pathogenesis for the present findings?\n      ANSWER: Thromboembolism\n', metadata={'source': 'test.txt'})

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)
doc_search = Chroma.from_documents(docs, embeddings)

In [ ]:
template = """
Use the following context (delimited by <ctx></ctx>) to answer the question. Given the symptoms, return only the disease / pathogen at position of <MASK>.
Return disease / pathogen name only, do not output anything else.
------
<ctx>
{context}
</ctx>
------
Question:
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [ ]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [ ]:
retrieval_chain = RetrievalQA.from_chain_type(llm,
                                              chain_type='stuff',
                                              retriever=doc_search.as_retriever(),
                                              chain_type_kwargs={
                                                  "prompt": prompt,
                                              })

In [ ]:
def invoke_query_chain(query):
    response = retrieval_chain.run(query)
    response=response.replace('<pad>', ' ')
    return response

In [ ]:
def find_sentence_with_word(index,df=df):
    allSentences=df['new_masked_text'][index]
    wrong_word=df['masked_word'][index]
    sentences = allSentences.split('\n')
    for i, sentence in enumerate(sentences):
      # print(i,":",sentence)
      if wrong_word in sentence:
        return i # Return the index
    # print("Not found")
    return -1

def get_response(index,df=df):
    actual_text=df['Sentences'][index]
    masked_ip=df['new_masked_text'][index]
    masked_word=df['masked_word'][index]
    corr_word= invoke_query_chain(masked_ip)
    corr_sent=masked_ip.replace("[MASK]",corr_word)
    error_flag=0
    if(masked_word.lower()!=corr_word.lower()):
        error_flag=1
    sent_id=find_sentence_with_word(index)

    print("ACTUAL TEXT :",actual_text)
    print("MASKED TEXT:",masked_ip)
    print("WRONG WORD :",masked_word)
    print("CORRECT WORD :",corr_word)
    print("CORRECT SENTENCE :",corr_sent)
    print("SENT ID :",sent_id)
    print("ERROR Flag :",error_flag)


> The provided context does not mention the disease / pathogen at position of <MASK>.

In [ ]:
response = invoke_query_chain('''0 A 29-year-old internal medicine resident presents to the emergency department with complaints of fevers, diarrhea, abdominal pain, and skin rash for 2 days.
1 He feels fatigued and has lost his appetite.
2 On further questioning, he says that he returned from his missionary trip to Brazil last week.
3 He is excited as he talks about his trip.
4 Besides a worthy clinical experience, he also enjoyed local outdoor activities, like swimming and rafting.
5 His past medical history is insignificant.
6 The blood pressure is
7 120/70 mm
8 Hg, the pulse is 100/min, and the temperature is 38.3 C (100.9 F).
9 On examination, there is a rash on the legs.
10 Patient's symptoms are suspected to be due to hepatitis A.
11 The rest of the examination is normal.
''')


In [ ]:
get_response(0)


#OUTPUT

# ACTUAL TEXT :0 A 29-year-old internal medicine resident presents to the emergency department with complaints of fevers, diarrhea, abdominal pain, and skin rash for 2 days.
# 1 He feels fatigued and has lost his appetite.
# 2 On further questioning, he says that he returned from his missionary trip to Brazil last week.
# 3 He is excited as he talks about his trip.
# 4 Besides a worthy clinical experience, he also enjoyed local outdoor activities, like swimming and rafting.
# 5 His past medical history is insignificant.
# 6 The blood pressure is
# 7 120/70 mm
# 8 Hg, the pulse is 100/min, and the temperature is 38.3 C (100.9 F).
# 9 On examination, there is a rash on the legs.
# 10 Patient's symptoms are suspected to be due to hepatitis A.
# 11 The rest of the examination is normal.

# MASKED TEXT:0 A 29-year-old internal medicine resident presents to the emergency department with complaints of fevers, diarrhea, abdominal pain, and skin rash for 2 days.
# 1 He feels fatigued and has lost his appetite.
# 2 On further questioning, he says that he returned from his missionary trip to Brazil last week.
# 3 He is excited as he talks about his trip.
# 4 Besides a worthy clinical experience, he also enjoyed local outdoor activities, like swimming and rafting.
# 5 His past medical history is insignificant.
# 6 The blood pressure is
# 7 120/70 mm
# 8 Hg, the pulse is 100/min, and the temperature is 38.3 C (100.9 F).
# 9 On examination, there is a rash on the legs.
# 10 Patient's symptoms are suspected to be due to [MASK]
# 11 The rest of the examination is normal.

# WRONG WORD : hepatitis A
# CORRECT WORD : Schistosoma mansoni
# CORRECT SENTENCE : Patient's symptoms are suspected to be due to Schistosoma mansoni
# SENT ID : 10
# ERROR Flag : 1
